# Clone impala mk2

## Initialization

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import pandas as pd
import pickle
import tensorflow as tf

In [ ]:
gpu_device = tf.config.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu_device, True)

To use gpu while training use this code snippit
```
with tf.device('/GPU:0'):
    # Create your TensorFlow model here
    model = tf.keras.Sequential([...])
```

In [9]:
np.random.seed(69)

In [10]:
# Getting the lyrics dataframe
df = pd.read_csv("corpuses\\tame impala lyrics.csv")
df = df.dropna()

# Joining all lines of lyrics into one single list
lyrics_corpus = []
for song_lryics in df['lyrics']:
    # Splitting the lyrics by line
    song_lryics = song_lryics.splitlines()
    
    #Removing empty lines
    song_lryics = list(filter(None, song_lryics))

    # Joining back lines
    song_lryics = '\n'.join(song_lryics)

    # Appending to songs corpus
    lyrics_corpus.append(song_lryics)

lyrics_corpus = '\n'.join(lyrics_corpus)
lyrics_corpus = lyrics_corpus.split('\n')
lyrics_corpus = [line + ' \n' for line in lyrics_corpus]

In [11]:
#To Load the tokenizer
with open('models\\tame_impala_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    total_words = tokenizer.num_words
    print(f"Total words: {total_words}")

Total words: 2000


In [12]:
# Load the data from the text file
input_sequences = np.loadtxt('corpuses\\input_sequences.txt', dtype=int)

# Reshape the loaded array to its original shape
original_shape = (18466, 200, 1)
input_sequences = input_sequences.reshape(original_shape)

sequence_len = len(input_sequences[0])
print(f"Input sequences length (of each sequence): {sequence_len}")

Input sequences length (of each sequence): 200


In [13]:
# For generating a sequence as input to generator
def generate_random_sequence():
    # Picking random length of lyric
    random_lyric_size = np.random.randint(1,200)
    
    # Generating till random length
    lyrics = [np.random.randint(0, 2000) for x in range(random_lyric_size)]

    # Padding with 0s
    padded_lyrics = [0 for _ in range(200 - random_lyric_size)]
    padded_lyrics.extend(lyrics)
    
    return np.array([padded_lyrics]).reshape(200,1)

## Data gathering

In [12]:
# genius api auth keys
file = open("genius_auth.json")
genius_auth = json.load(file)
file.close()

In [13]:
from lyricsgenius import Genius

# API Client
genius = Genius(genius_auth['client_access_token'],
                verbose=False,
                skip_non_songs=True, 
                excluded_terms=["(Remix)", "(Live)"], 
                remove_section_headers=True)

In [14]:
# Getting all the songs
songs = genius.search_artist(artist_name="Tame Impala", max_songs=None).songs

In [15]:
songs

[Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist, ...),
 Song(id, artist

### Turning the obtained information into a pandas dataframe

In [16]:
songs[0].lyrics

'159 ContributorsTranslationsFrançaisTürkçeEspañolPortuguêsItalianoDeutschThe Less I Know the Better Lyrics\nSomeone said they left together\nI ran out the door to get her\nShe was holding hands with Trevor\nNot the greatest feeling ever\nSaid, "Pull yourself together\nYou should try your luck with Heather"\nThen I heard they slept together\nOh, the less I know the better\nThe less I know the better\n\nOh, my love, can\'t you see yourself by my side?\nNo surprise, when you\'re on his shoulder like every night\nOh, my love, can\'t you see that you\'re on my mind?\nDon\'t suppose you could convince your lover to change his mind\nSo goodbye\n\nShe said, "It\'s not now or never\nWait ten years, we\'ll be together"\nI said, "Better late than never\nJust don\'t make me wait forever"\nDon\'t make me wait forever\nDon\'t make me wait forever\nYou might also like\nOh, my love, can\'t you see yourself by my side?\nI don\'t suppose you could convince your lover to change his mind\n\nI was doing f

In [43]:
data = {
    "id":[],
    "artist":[],
    "title":[],
    "lyrics":[]
}

# Iterating through the list
for song in songs:
    if not song:
        continue
    data["id"].append(song.id)
    data["artist"].append(song.artist)
    data["title"].append(song.title)

    # Preprocessing the lyrics to remove watermarks
    song_lyrics = song.lyrics
    ## Getting rid of the first line (shows song metadata)
    song_lyrics = re.sub(r'^.*?Lyrics','',song_lyrics)
    ## Removing the watermark in the last line
    song_lyrics = re.sub(r'(You might also like)?(\d*)?Embed','',song_lyrics, flags=re.IGNORECASE)
    ## Splitting the lyrics by line
    song_lyrics = song_lyrics.splitlines()
    ##Removing empty lines
    song_lyrics = list(filter(None, song_lyrics))
    ## Joining back lines
    song_lyrics = '\n'.join(song_lyrics)

    data["lyrics"].append(song_lyrics)
    

# Turning into dataframe
df = pd.DataFrame(data)
df.to_csv('corpuses\\tame impala lyrics.csv', index=False)

In [41]:
df = pd.read_csv("corpuses\\tame impala lyrics.csv")
df.head()

,id,artist,title,lyrics
0,2165830,Tame Impala,The Less I Know the Better,Someone said they left together\nI ran out the...
1,2165813,Tame Impala,"New Person, Same Old Mistakes","I can just hear them now\n""How could you let u..."
2,721026,Tame Impala,Let It Happen,"It's always around me, all this noise\nBut not..."
3,2165828,Tame Impala,Yes I’m Changing,"I was raging, it was late\nIn the world my dem..."
4,94120,Tame Impala,Feels Like We Only Go Backwards,"It feels like I only go backwards, baby\nEvery..."


## Tokenizer creation

In [50]:
# Joining all lines of lyrics into one single list
lyrics_corpus = []
for song_lryics in df['lyrics']:
    # Splitting the lyrics by line
    song_lryics = song_lryics.splitlines()
    
    #Removing empty lines
    song_lryics = list(filter(None, song_lryics))

    # Joining back lines
    song_lryics = '\n'.join(song_lryics)

    # Appending to songs corpus
    lyrics_corpus.append(song_lryics)

lyrics_corpus = '\n'.join(lyrics_corpus)
lyrics_corpus = lyrics_corpus.split('\n')
lyrics_corpus = [line + ' \n' for line in lyrics_corpus]

In [53]:
lyrics_corpus[:3]

['Someone said they left together \n',
 'I ran out the door to get her \n',
 'She was holding hands with Trevor \n']

In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Making tokenizer
tokenizer = Tokenizer(num_words=2000, oov_token="OOV",
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t')

# Fitting corpus
tokenizer.fit_on_texts(lyrics_corpus)


total_words = tokenizer.num_words
print(f"Total words in tokenizer:{total_words}")

# Saving tokenizer
with open('models\\tame_impala_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Total words in tokenizer:2000


In [58]:
#To Load the tokenizer
with open('models\\tame_impala_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    total_words = tokenizer.num_words

## Sequence creation

In [39]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
# Converting the corpus into sequences
sequences = []
for line in lyrics_corpus:
    tokenized_line = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokenized_line)):
        n_gram_sequence = tokenized_line[:i]
        sequences.append(n_gram_sequence)

In [42]:
# Padding sequences
max_sequence_len = 200
num_sequences = len(sequences)
input_sequences = np.array(pad_sequences(sequences, 
                                   maxlen=max_sequence_len,
                                   padding = 'pre')).reshape(num_sequences, 200, 1)

In [48]:
# Saving the input sequences
filename = "corpuses\\input_sequences.txt"

# Use numpy.savetxt to save the array to a text file
np.savetxt(filename, input_sequences.reshape(-1, input_sequences.shape[-1]), fmt='%d')

## Model architecture

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Dropout, Bidirectional, LSTM, Reshape

### Generator

In [26]:
def build_generator():
    model = Sequential()
    
    # This model is very similar to the model used in 'clone impala mk1'
    # model.add(Embedding(input_dim=total_words, output_dim=64, 
    #                     input_length=sequence_len))
    model.add(Dense(8*8*256, input_dim=256))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.2))
    model.add(Dense(total_words, activation='softmax'))
    # model.add(Dense(200, activation='relu'))

    return model

In [27]:
def build_generator():
    model = Sequential()
    
    model.add(Dense(64, input_dim=64))
    model.add(Reshape())
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(200))

    return model

In [28]:
generator = build_generator()

ValueError: Input 0 of layer "bidirectional_6" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [62]:
generator.predict(tf.random.normal((256, 1)))

8/8 [==============================] - 13s 12ms/step


array([[ 1.4941040e-03,  1.1103499e-03,  8.2597136e-04, ...,
        -1.0661436e-04, -1.8723702e-04, -7.1735657e-04],
       [ 1.4941040e-03,  1.1103499e-03,  8.2597136e-04, ...,
        -1.0661436e-04, -1.8723702e-04, -7.1735657e-04],
       [ 1.4941040e-03,  1.1103499e-03,  8.2597136e-04, ...,
        -1.0661436e-04, -1.8723702e-04, -7.1735657e-04],
       ...,
       [-9.9160956e-05,  1.4566074e-03,  1.5361075e-03, ...,
         1.4107404e-03, -2.9172984e-04, -4.2169951e-05],
       [ 1.4941040e-03,  1.1103499e-03,  8.2597136e-04, ...,
        -1.0661436e-04, -1.8723702e-04, -7.1735657e-04],
       [ 1.4941040e-03,  1.1103499e-03,  8.2597136e-04, ...,
        -1.0661436e-04, -1.8723702e-04, -7.1735657e-04]], dtype=float32)

In [59]:
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 64)           128000    
                                                                 
 dropout_4 (Dropout)         (None, 200, 64)           0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 200, 128)         66048     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 200)              

In [15]:
# Generating some new lyrics
next_words = 25
seed_text = "Strictly speaking"
for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=sequence_len, padding='pre')
        predicted = np.argmax(generator.predict(token_list, verbose=0), axis=-1)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
print(seed_text)

Strictly speaking insane loss loss loss loss                    


The untrained model produces gibbrish

### Discriminator

The discriminator will evaluate the probabilities for the next word produced by the generator

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LeakyReLU, Conv1D, Flatten

In [17]:
def build_discriminator():
    model = Sequential()
    # First Conv Block
    model.add(Conv1D(32, 5, input_shape = (2000,1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Second Conv Block
    model.add(Conv1D(64, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Third Conv Block
    model.add(Conv1D(128, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Fourth Conv Block
    model.add(Conv1D(256, 5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Flatten then pass to dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [18]:
discriminator = build_discriminator()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1996, 32)          192       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1996, 32)          0         
                                                                 
 dropout_2 (Dropout)         (None, 1996, 32)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1992, 64)          10304     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 1992, 64)          0         
                                                                 
 dropout_3 (Dropout)         (None, 1992, 64)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1988, 128)        

In [20]:
tokens = [[np.random.randint(0,2000) for x in range (200)]]
probs = generator.predict(tokens)
probs

1/1 [==============================] - 0s 58ms/step


array([[0.00050096, 0.00050081, 0.00050014, ..., 0.00049978, 0.00050044,
        0.00049993]], dtype=float32)

In [25]:
discriminator.predict(probs)

1/1 [==============================] - 4s 4s/step


array([[0.50000167]], dtype=float32)

As we can see, the untrained discriminator is unsure weather this is a real lyric or not

## Training loop

### Setting up losses and optimizers

In [26]:
# Adam optimizer for both
from tensorflow.keras.optimizers import Adam

# Binary cross entropy for both
from tensorflow.keras.losses import BinaryCrossentropy

In [27]:
g_opt = Adam(learning_rate=.0001)
d_opt = Adam(learning_rate=.00001)

g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

### Building subclass model

In [28]:
from tensorflow.keras.models import Model

In [40]:
class LyricsGAN(Model):
    def __init__(self, generator, discriminator, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.generator = generator
        self.discriminator = discriminator

    def compile(self, g_opt, d_opt, g_loss, d_loss,  *args, **kwargs):
        super().compile( *args, **kwargs)

        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss
    
    def train_step(self, batch):
        # Get the data 
        real_lyrics = batch
        fake_lyrics = self.generator(generate_random_sequence(), 
                                     training=False)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_lyrics, training=True) 
            yhat_fake = self.discriminator(fake_lyrics, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            # Create labels for real and fakes images
            y_realfake = tf.concat([tf.zeros_like(yhat_real), 
                                    tf.ones_like(yhat_fake)], axis=0)
            
            # Add some noise to the TRUE outputs
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation - nn learn 
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        # Train the generator 
        with tf.GradientTape() as g_tape:
            # Generate some new images
            gen_lyrics = self.generator(generate_random_tokens(), 
                                        training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_lyrics, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [41]:
# Create instance of subclassed model
lyricsgan = LyricsGAN(generator, discriminator)

In [42]:
# Compile the model
lyricsgan.compile(g_opt, d_opt, g_loss, d_loss)

### Build callback

It is only to store the generated output of the model as we are training, not necessary to run

In [9]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim,1))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('images', f'generated_img_{epoch}_{i}.png')) 

### Train

In [43]:
# Recommend 2000 epochs
# hist = lyricsgan.fit(ds, epochs=20, callbacks=[ModelMonitor()])
hist = lyricsgan.fit(input_sequences, epochs=20)

Epoch 1/20


ValueError: in user code:

    File "c:\Users\sanya\Documents\Code projects\clone-impala-mk2\env\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sanya\Documents\Code projects\clone-impala-mk2\env\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sanya\Documents\Code projects\clone-impala-mk2\env\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\sanya\AppData\Local\Temp\ipykernel_8900\2911947785.py", line 25, in train_step
        yhat_real = self.discriminator(real_lyrics, training=True)
    File "c:\Users\sanya\Documents\Code projects\clone-impala-mk2\env\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\

In [39]:
len(input_sequences[0])

199

### Review Performance

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['d_loss'], label='d_loss')
plt.plot(hist.history['g_loss'], label='g_loss')
plt.legend()
plt.show()

## Generating the output